# RET-CLIP Complete Training & Evaluation Pipeline

This notebook implements a comprehensive pipeline for training and evaluating **RET-CLIP** (Retinal-Contrastive Language-Image Pre-training), a foundation model for retinal fundus image analysis.

---

## 📚 Background

**CLIP (Contrastive Language-Image Pre-training)** [Radford et al., 2021] revolutionized vision-language learning by jointly training image and text encoders using contrastive learning. RET-CLIP adapts this approach for medical imaging, specifically retinal fundus photography.

### Key Concepts:

1. **Contrastive Learning**: Train the model to align images with their corresponding text descriptions while pushing apart mismatched pairs
2. **Dual Encoders**: 
   - **Vision Encoder**: ViT-B-16 (Vision Transformer) for fundus images
   - **Text Encoder**: PubMedBERT for clinical descriptions
3. **Zero-Shot Classification**: Use text prompts at inference time without task-specific fine-tuning
4. **Linear Probing**: Evaluate learned representations by training a simple classifier on frozen features

### References:
- Radford et al. (2021). "Learning Transferable Visual Models From Natural Language Supervision." ICML.
- Dosovitskiy et al. (2020). "An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale." ICLR.
- Gu et al. (2021). "Domain-Specific Language Model Pretraining for Biomedical Natural Language Processing." ACM TIST.

---

## 🔄 Pipeline Overview

```
┌─────────────────────────────────────┐
│  1. Setup & Environment             │  ← Dependencies, GPU, Drive
└─────────────────────────────────────┘
                ↓
┌─────────────────────────────────────┐
│  2. Load Pre-Generated Prompts      │  ← CSV with clinical descriptions
└─────────────────────────────────────┘
                ↓
┌─────────────────────────────────────┐
│  3. Convert CSV → TSV + JSONL       │  ← Download & encode images
└─────────────────────────────────────┘
                ↓
┌─────────────────────────────────────┐
│  4. Build LMDB Database             │  ← Efficient storage format
└─────────────────────────────────────┘
                ↓
┌─────────────────────────────────────┐
│  5. Train RET-CLIP (10 epochs)      │  ← Contrastive learning
└─────────────────────────────────────┘
                ↓
┌─────────────────────────────────────┐
│  6. Zero-Shot Evaluation            │  ← Vision-language alignment
└─────────────────────────────────────┘
                ↓
┌─────────────────────────────────────┐
│  7. Linear Probing Evaluation       │  ← Feature quality assessment
└─────────────────────────────────────┘
                ↓
┌─────────────────────────────────────┐
│  8. Generate Final Report           │  ← Metrics & comparison
└─────────────────────────────────────┘
```

---

## ✅ Prerequisites

**Before running this notebook:**

1. **Generate Prompts**: Run `generate_retclip_prompts.ipynb` to create clinical descriptions
2. **Upload CSV Files** to Google Drive:
   - `retclip_prompts_full.csv` (12,989 train samples)
   - `retclip_prompts_test.csv` (3,253 test samples)
3. **HuggingFace Token**: Get from https://huggingface.co/settings/tokens
4. **GPU**: Use Google Colab with A100 GPU (Runtime → Change runtime type → A100 GPU)

---

## ⏱️ Estimated Runtime

| Step | Duration (A100 GPU) |
|------|--------------------| 
| Setup | ~5 minutes |
| CSV → TSV Conversion | ~2-3 hours |
| LMDB Building | ~30 minutes |
| RET-CLIP Training | ~6-8 hours |
| Zero-Shot Evaluation | ~15 minutes |
| Linear Probing | ~20 minutes |
| **Total** | **~9-12 hours** |

💾 **All artifacts are automatically saved to Google Drive throughout the pipeline.**

---

**Let's begin!**

# STEP 1: Setup & Environment

## Overview

This step prepares the environment by:
1. Checking GPU availability
2. Mounting Google Drive for persistent storage
3. Installing required Python packages
4. Cloning the RET-CLIP repository
5. Creating the folder structure

## Required Packages

- **ftfy, regex, tqdm**: Text processing and progress bars
- **CLIP**: OpenAI's CLIP implementation (vision-language model base)
- **transformers**: HuggingFace library for PubMedBERT
- **lmdb**: Lightning Memory-Mapped Database for efficient data loading
- **datasets, huggingface-hub**: Dataset downloading and authentication
- **scikit-learn, matplotlib, seaborn**: Evaluation and visualization

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted successfully")

In [ ]:
# Create Drive folder structure
import os
from pathlib import Path

DRIVE_BASE = "/content/drive/MyDrive/RET-CLIP"
DRIVE_DATA = f"{DRIVE_BASE}/data"
DRIVE_LMDB = f"{DRIVE_BASE}/lmdb"
DRIVE_CHECKPOINTS = f"{DRIVE_BASE}/checkpoints"
DRIVE_RESULTS = f"{DRIVE_BASE}/results"

# Create directories
for path in [DRIVE_BASE, DRIVE_DATA, DRIVE_LMDB, DRIVE_CHECKPOINTS, DRIVE_RESULTS]:
    os.makedirs(path, exist_ok=True)

print("✅ Created Drive folder structure:")
print(f"   {DRIVE_BASE}/")
print(f"   ├── data/")
print(f"   ├── lmdb/")
print(f"   ├── checkpoints/")
print(f"   └── results/")

In [ ]:
# Install dependencies
print("Installing dependencies...\n")

!pip install -q ftfy regex tqdm
!pip install -q git+https://github.com/openai/CLIP.git
!pip install -q transformers
!pip install -q lmdb
!pip install -q datasets huggingface-hub
!pip install -q scikit-learn matplotlib seaborn

print("\n✅ All dependencies installed")

## ⚠️ Important: RET-CLIP Repository Fixes

The original RET-CLIP repository requires several fixes to work with English text and our dataset format. Before running this notebook, you need to apply these changes to your forked repository.

### Required Fixes:

#### 1. **Fix Chinese BERT → English PubMedBERT Issue**

**File**: `RET_CLIP/clip/model.py`

**Problem**: The original code forces Chinese BERT tokenizer and config, breaking English models.

**Fix**: In the `CLIP.__init__()` method, find this section:
```python
if isinstance(text_cfg, dict):
    # Force Chinese config (WRONG!)
    text_cfg = CLIPTextCfg(**text_cfg)
```

**Replace with**:
```python
if isinstance(text_cfg, dict):
    # Keep the config as-is for any BERT model
    text_cfg = CLIPTextCfg(**text_cfg)
```

Also ensure the tokenizer initialization uses the correct model name from config.

---

#### 2. **Fix Base64 Encoding (URL-safe)**

**File**: `RET_CLIP/preprocess/build_lmdb_dataset_for_RET-CLIP.py`

**Issue**: The LMDB builder must use URL-safe base64 decoding to match our encoding.

**Current code** (line ~95):
```python
img_str = base64.b64decode(img_l)
```

**Fix**:
```python
img_str = base64.urlsafe_b64decode(img_l)
```

Apply this to both `img_l` and `img_r` decoding.

---

#### 3. **Fix TSV Format (3-column support)**

**File**: `RET_CLIP/preprocess/build_lmdb_dataset_for_RET-CLIP.py`

The code already expects 3 columns, but make sure it matches:
```python
patient_id, img_l, img_r = line.split("\t")
```

This is correct - our preprocessing creates 3-column TSV files.

---

#### 4. **Fix Model Loading (DDP compatibility)**

**Files**: Evaluation scripts

When loading checkpoints saved with DistributedDataParallel, strip the `module.` prefix:

```python
checkpoint = torch.load(checkpoint_path)
state_dict = checkpoint['state_dict']

# Remove 'module.' prefix from DDP
new_state_dict = {}
for k, v in state_dict.items():
    name = k.replace('module.', '')
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)
```

---

#### 5. **Add English BERT Model Configurations**

**Directory**: `RET_CLIP/clip/model_configs/`

**Issue**: The repository only includes Chinese BERT configs. We need to add English BERT model configs for our medical text.

**Required config files** (create these JSON files):

**a) `microsoft-BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext.json`** (Medical domain):
```json
{
    "vocab_size": 30522,
    "text_attention_probs_dropout_prob": 0.1,
    "text_hidden_act": "gelu",
    "text_hidden_dropout_prob": 0.1,
    "text_hidden_size": 768,
    "text_initializer_range": 0.02,
    "text_intermediate_size": 3072,
    "text_max_position_embeddings": 512,
    "text_num_attention_heads": 12,
    "text_num_hidden_layers": 12,
    "text_type_vocab_size": 2
}
```

**b) `bert-base-uncased.json`** (General English):
```json
{
    "vocab_size": 30522,
    "text_attention_probs_dropout_prob": 0.1,
    "text_hidden_act": "gelu",
    "text_hidden_dropout_prob": 0.1,
    "text_hidden_size": 768,
    "text_initializer_range": 0.02,
    "text_intermediate_size": 3072,
    "text_max_position_embeddings": 512,
    "text_num_attention_heads": 12,
    "text_num_hidden_layers": 12,
    "text_type_vocab_size": 2
}
```

**c) `dmis-lab-biobert-base-cased-v1.1.json`** (Biomedical domain):
```json
{
    "vocab_size": 28996,
    "text_attention_probs_dropout_prob": 0.1,
    "text_hidden_act": "gelu",
    "text_hidden_dropout_prob": 0.1,
    "text_hidden_size": 768,
    "text_initializer_range": 0.02,
    "text_intermediate_size": 3072,
    "text_max_position_embeddings": 512,
    "text_num_attention_heads": 12,
    "text_num_hidden_layers": 12,
    "text_type_vocab_size": 2
}
```

**Where to add these**: Place all three JSON files in the `RET_CLIP/clip/model_configs/` directory of your forked repository.

---

### Quick Setup Guide:

1. **Fork the RET-CLIP repository**: https://github.com/SUSTechBruce/RET-CLIP
2. **Apply the fixes above** to your fork
3. **Add the three BERT config files** to `RET_CLIP/clip/model_configs/`
4. **Push to GitHub**
5. **Update the clone command** in the next cell with your GitHub username

Alternatively, you can use our pre-fixed repository (if available).

In [ ]:
# Clone this repository (contains fixed RET-CLIP in retclip/)
import os
import shutil

# This repository contains the fixed RET-CLIP in retclip/
REPO_URL = "https://github.com/FahadAlothman-fsd/retclip-english.git"

if not os.path.exists('/content/retclip_repo'):
    print(f"Cloning repository from {REPO_URL}...")
    !git clone {REPO_URL} /content/retclip_repo
    print("✅ Repository cloned")
else:
    print("✅ Repository already exists")

# Copy retclip to /content/retclip for use in the pipeline
if not os.path.exists('/content/retclip'):
    print("\nCopying fixed RET-CLIP from retclip...")
    shutil.copytree('/content/retclip_repo/retclip', '/content/retclip')
    print("✅ Copied to /content/retclip")
else:
    print("✅ /content/retclip already exists")

# Add to Python path
import sys
sys.path.insert(0, '/content/retclip')

print("\n✅ RET-CLIP repository ready")
print("\n" + "="*80)
print("VERIFY: Using fixed RET-CLIP with:")
print("="*80)
print("  1. ✓ English BERT configs (PubMedBERT, BERT-base, BioBERT)")
print("  2. ✓ English BERT models in choices arrays")
print("  3. ✓ DDP checkpoint loading with 'module.' prefix stripping")
print("  4. ✓ URL-safe base64 encoding/decoding")
print("  5. ✓ 3-column TSV format support")
print("\nLocation: /content/retclip (copied from retclip/)")
print(f"Source: {REPO_URL}")

In [ ]:
# Save setup log
import datetime

setup_log = f"""
RET-CLIP Pipeline Setup Log
===========================
Date: {datetime.datetime.now()}
GPU: Checked via nvidia-smi
Drive: Mounted at /content/drive
Repository: Cloned to /content/retclip
Dependencies: Installed

Drive Structure:
{DRIVE_BASE}/
├── data/
├── lmdb/
├── checkpoints/
└── results/

Status: ✅ Setup Complete
"""

with open(f"{DRIVE_BASE}/setup_log.txt", "w") as f:
    f.write(setup_log)

print(setup_log)
print(f"\n✅ Setup log saved to {DRIVE_BASE}/setup_log.txt")

# STEP 2: Load Pre-Generated Prompts

Load the CSV files generated by `generate_retclip_prompts.ipynb`.

**Important:** Make sure you've uploaded these files to your Google Drive first!

In [ ]:
# ========================================
# CONFIGURATION
# ========================================

# TEST MODE: Toggle for quick testing vs full pipeline
TEST_MODE = True  # Set to False for full training

if TEST_MODE:
    TRAIN_SAMPLE_SIZE = 100  # Use 100 samples for testing
    TEST_SAMPLE_SIZE = 50    # Use 50 samples for testing
    NUM_EPOCHS_CONFIG = 2    # Train for only 2 epochs in test mode
    print("⚠️ TEST MODE ENABLED - Using small sample for quick testing")
else:
    TRAIN_SAMPLE_SIZE = None  # Use all samples
    TEST_SAMPLE_SIZE = None   # Use all samples
    NUM_EPOCHS_CONFIG = 10    # Full training: 10 epochs
    print("✅ FULL MODE - Using complete dataset")

# TEXT ENCODER COMPARISON: Toggle to compare multiple text encoders
# In TEST_MODE: ~1.5-2 hours total (3 encoders × ~30 min each)
# In FULL_MODE: ~18-24 hours total (3 encoders × ~6-8 hours each)
RUN_TEXT_ENCODER_COMPARISON = True  # Set to False to use only PubMedBERT
print(f"Text Encoder Comparison: {'✅ ENABLED' if RUN_TEXT_ENCODER_COMPARISON else '❌ DISABLED (single encoder only)'}")

# Configure text encoders to compare (if comparison enabled)
TEXT_ENCODERS = [
    {
        "name": "PubMedBERT",
        "model_id": "microsoft-BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
        "description": "Medical domain-specific (PubMed abstracts)"
    },
    {
        "name": "BERT-base",
        "model_id": "bert-base-uncased",
        "description": "General English (Wikipedia + BookCorpus)"
    },
    {
        "name": "BioBERT",
        "model_id": "dmis-lab-biobert-base-cased-v1.1",
        "description": "Biomedical domain (PubMed + PMC)"
    }
]

if RUN_TEXT_ENCODER_COMPARISON:
    print("\n📋 Text Encoders to Compare:")
    for encoder in TEXT_ENCODERS:
        print(f"  • {encoder['name']}: {encoder['description']}")

# ========================================
# SECRETS MANAGEMENT
# ========================================

# Try to load from Colab secrets first, fallback to manual entry
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("✅ HuggingFace token loaded from Colab secrets")
except:
    HF_TOKEN = ""  # Set manually if not using secrets
    print("⚠️ HF_TOKEN not found in secrets - please set manually below or add to Colab secrets")
    print("   To add secrets: Click 🔑 in left sidebar → Add HF_TOKEN")

# Manual override (leave empty if using secrets)
HF_TOKEN_OVERRIDE = ""  # Optionally set your token here
if HF_TOKEN_OVERRIDE:
    HF_TOKEN = HF_TOKEN_OVERRIDE
    print("✅ Using manually set HF_TOKEN")

# ========================================
# PATHS AND CONFIGURATION
# ========================================

# Load CSV files from cloned repo (faster than uploading to Drive!)
TRAIN_CSV_PATH = "/content/retclip_repo/content/retclip_prompts_full.csv"
TEST_CSV_PATH = "/content/retclip_repo/content/retclip_prompts_test.csv"

# HuggingFace configuration
HF_DATASET_NAME = "Peacein/color-fundus-eye"

# Training configuration
IMAGE_SIZE = 224
CHECKPOINT_INTERVAL = 100  # Save progress every N samples

print("\n" + "="*80)
print("CONFIGURATION SUMMARY")
print("="*80)
print(f"  Mode: {'TEST (small sample)' if TEST_MODE else 'FULL (complete dataset)'}")
if TEST_MODE:
    print(f"  Train samples: {TRAIN_SAMPLE_SIZE}")
    print(f"  Test samples: {TEST_SAMPLE_SIZE}")
    print(f"  Epochs: {NUM_EPOCHS_CONFIG}")
print(f"  Train CSV: {TRAIN_CSV_PATH}")
print(f"  Test CSV: {TEST_CSV_PATH}")
print(f"  CSV Source: Loaded from GitHub repo (no Drive upload needed!)")
print(f"  HuggingFace Dataset: {HF_DATASET_NAME}")
print(f"  Image Size: {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"  HF Token: {'✅ Set' if HF_TOKEN else '❌ Not set - please update'}")

In [ ]:
import pandas as pd

# Load train CSV
if not os.path.exists(TRAIN_CSV_PATH):
    raise FileNotFoundError(f"Train CSV not found at {TRAIN_CSV_PATH}. Please upload it to Google Drive.")

train_df = pd.read_csv(TRAIN_CSV_PATH)
print(f"✅ Loaded train CSV: {len(train_df)} samples (before sampling)")

# Apply TEST_MODE sampling if enabled
if TEST_MODE and TRAIN_SAMPLE_SIZE:
    train_df = train_df.head(TRAIN_SAMPLE_SIZE)
    print(f"⚠️ TEST MODE: Sampled {len(train_df)} training samples")

print(f"   Final size: {len(train_df)} samples")
print(f"   Columns: {train_df.columns.tolist()}")
print(f"\nFirst 3 rows:")
display(train_df.head(3))

# Validate structure
required_columns = ['dataset_index', 'label', 'prompt']
for col in required_columns:
    if col not in train_df.columns:
        raise ValueError(f"Train CSV missing required column: {col}")

print(f"\n✅ Train CSV validation passed")

In [ ]:
# Load test CSV
if not os.path.exists(TEST_CSV_PATH):
    raise FileNotFoundError(f"Test CSV not found at {TEST_CSV_PATH}. Please upload it to Google Drive.")

test_df = pd.read_csv(TEST_CSV_PATH)
print(f"✅ Loaded test CSV: {len(test_df)} samples (before sampling)")

# Apply TEST_MODE sampling if enabled
if TEST_MODE and TEST_SAMPLE_SIZE:
    test_df = test_df.head(TEST_SAMPLE_SIZE)
    print(f"⚠️ TEST MODE: Sampled {len(test_df)} test samples")

print(f"   Final size: {len(test_df)} samples")
print(f"   Columns: {test_df.columns.tolist()}")
print(f"\nFirst 3 rows:")
display(test_df.head(3))

# Validate structure
for col in required_columns:
    if col not in test_df.columns:
        raise ValueError(f"Test CSV missing required column: {col}")

print(f"\n✅ Test CSV validation passed")

In [ ]:
# Summary statistics
print("\n" + "="*80)
print("DATASET SUMMARY")
print("="*80)

print(f"\nTrain Set:")
print(f"  Total samples: {len(train_df)}")
print(f"  Unique labels: {train_df['label'].nunique()}")
print(f"  Label distribution:")
for label, count in train_df['label'].value_counts().items():
    print(f"    {label}: {count}")

print(f"\nTest Set:")
print(f"  Total samples: {len(test_df)}")
print(f"  Unique labels: {test_df['label'].nunique()}")
print(f"  Label distribution:")
for label, count in test_df['label'].value_counts().items():
    print(f"    {label}: {count}")

# STEP 3: Convert CSV → TSV + JSONL

Download images from HuggingFace, encode as base64, and create training files.

**Output:**
- TSV: `patient_id\tleft_img_base64\tright_img_base64` (3 columns)
- JSONL: `{"text_id": 0, "text": "prompt", "image_ids": ["patient_000000"]}`

In [ ]:
import base64
import json
import time
from io import BytesIO
from PIL import Image
from tqdm.notebook import tqdm
from datasets import load_dataset
from huggingface_hub import login

# Authenticate with HuggingFace
if HF_TOKEN:
    login(token=HF_TOKEN)
    print("✅ Authenticated with HuggingFace")
else:
    print("⚠️ No HF_TOKEN set - download may fail")

def image_to_base64(image, size=224):
    """Convert PIL Image to URL-safe base64 string with resizing"""
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    # Resize image
    image = image.resize((size, size), Image.BICUBIC)
    
    # Convert to base64 (URL-safe encoding)
    buffered = BytesIO()
    image.save(buffered, format="JPEG", quality=95)
    img_str = base64.urlsafe_b64encode(buffered.getvalue()).decode()
    return img_str

print("\n✅ Helper functions defined")

In [ ]:
def convert_csv_to_tsv_jsonl(csv_df, split_name, output_dir, hf_dataset_name, image_size=224, checkpoint_interval=100):
    """
    Convert CSV with prompts to TSV (images) + JSONL (text annotations)
    
    Args:
        csv_df: DataFrame with columns [dataset_index, label, prompt]
        split_name: 'train' or 'test'
        output_dir: Where to save TSV and JSONL files
        hf_dataset_name: HuggingFace dataset name
        image_size: Resize images to this size
        checkpoint_interval: Save checkpoint every N samples
    """
    tsv_path = f"{output_dir}/{split_name}_imgs.tsv"
    jsonl_path = f"{output_dir}/{split_name}_texts.jsonl"
    checkpoint_path = f"{output_dir}/{split_name}_checkpoint.json"
    
    # Load checkpoint if exists
    processed_indices = set()
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'r') as f:
            checkpoint = json.load(f)
            processed_indices = set(checkpoint.get('processed_indices', []))
        print(f"✅ Resuming from checkpoint: {len(processed_indices)} samples already processed")
    
    # Load HuggingFace dataset in streaming mode
    print(f"\nLoading HuggingFace dataset: {hf_dataset_name} ({split_name})")
    hf_dataset = load_dataset(hf_dataset_name, split=split_name, streaming=True)
    
    # Open output files (append mode if resuming)
    mode = 'a' if processed_indices else 'w'
    tsv_file = open(tsv_path, mode, encoding='utf-8')
    jsonl_file = open(jsonl_path, mode, encoding='utf-8')
    
    # Track unique patients to avoid duplicates
    unique_patients_written = set()
    if processed_indices and os.path.exists(tsv_path):
        with open(tsv_path, 'r', encoding='utf-8') as f:
            for line in f:
                patient_id = line.split('\t')[0]
                unique_patients_written.add(patient_id)
    
    print(f"\nProcessing {len(csv_df)} samples...")
    print(f"Checkpoint interval: {checkpoint_interval}")
    
    # Create iterator from streaming dataset
    dataset_iter = iter(hf_dataset)
    current_stream_index = 0
    
    # Process each row in CSV
    for csv_idx, row in tqdm(csv_df.iterrows(), total=len(csv_df), desc=f"Converting {split_name}"):
        dataset_index = int(row['dataset_index'])
        label = row['label']
        prompt = row['prompt']
        
        # Skip if already processed
        if dataset_index in processed_indices:
            continue
        
        # Generate patient ID
        patient_id = f"patient_{dataset_index:06d}"
        
        try:
            # Stream forward to the correct index
            while current_stream_index < dataset_index:
                try:
                    next(dataset_iter)
                    current_stream_index += 1
                except StopIteration:
                    print(f"\n⚠️ Reached end of stream at index {current_stream_index}")
                    break
            
            # Get the sample at the correct index
            if current_stream_index == dataset_index:
                sample = next(dataset_iter)
                current_stream_index += 1
            else:
                print(f"⚠️ Could not reach index {dataset_index}, skipping")
                continue
            
            # Extract and encode image
            if 'image' in sample:
                image = sample['image']
                # For RET-CLIP binocular architecture, duplicate for both eyes
                img_b64 = image_to_base64(image, image_size)
                left_b64 = img_b64
                right_b64 = img_b64  # Same image for both eyes
            else:
                print(f"⚠️ No image field for index {dataset_index}")
                continue
            
            # Write to TSV (only if patient not already written)
            if patient_id not in unique_patients_written:
                tsv_file.write(f"{patient_id}\t{left_b64}\t{right_b64}\n")
                unique_patients_written.add(patient_id)
            
            # Write to JSONL
            text_annotation = {
                "text_id": csv_idx,
                "text": prompt,
                "image_ids": [patient_id]
            }
            jsonl_file.write(json.dumps(text_annotation, ensure_ascii=False) + '\n')
            
            # Update checkpoint
            processed_indices.add(dataset_index)
            
            # Save checkpoint periodically
            if (csv_idx + 1) % checkpoint_interval == 0:
                with open(checkpoint_path, 'w') as f:
                    json.dump({
                        'processed_indices': list(processed_indices),
                        'last_index': dataset_index
                    }, f)
                tsv_file.flush()
                jsonl_file.flush()
            
            # Small delay to avoid rate limits
            time.sleep(0.01)
            
        except StopIteration:
            print(f"\n⚠️ Reached end of dataset stream at index {dataset_index}")
            break
        except Exception as e:
            print(f"\n❌ Error processing index {dataset_index}: {e}")
            # Save checkpoint on error
            with open(checkpoint_path, 'w') as f:
                json.dump({
                    'processed_indices': list(processed_indices),
                    'last_index': dataset_index
                }, f)
            tsv_file.flush()
            jsonl_file.flush()
            continue
    
    # Close files
    tsv_file.close()
    jsonl_file.close()
    
    # Clean up checkpoint after successful completion
    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)
    
    print(f"\n✅ Successfully created:")
    print(f"   TSV: {tsv_path} ({len(unique_patients_written)} unique patients)")
    print(f"   JSONL: {jsonl_path} ({len(processed_indices)} text-image pairs)")
    
    return tsv_path, jsonl_path

print("✅ Conversion function defined")

In [ ]:
# Convert train set
print("="*80)
print("Converting TRAIN set to TSV + JSONL")
print("="*80)

train_tsv, train_jsonl = convert_csv_to_tsv_jsonl(
    csv_df=train_df,
    split_name='train',
    output_dir=DRIVE_DATA,
    hf_dataset_name=HF_DATASET_NAME,
    image_size=IMAGE_SIZE,
    checkpoint_interval=CHECKPOINT_INTERVAL
)

In [ ]:
# Convert test set
print("\n" + "="*80)
print("Converting TEST set to TSV + JSONL")
print("="*80)

test_tsv, test_jsonl = convert_csv_to_tsv_jsonl(
    csv_df=test_df,
    split_name='test',
    output_dir=DRIVE_DATA,
    hf_dataset_name=HF_DATASET_NAME,
    image_size=IMAGE_SIZE,
    checkpoint_interval=CHECKPOINT_INTERVAL
)

In [ ]:
# Validate TSV format
print("\n" + "="*80)
print("Validating TSV files")
print("="*80)

for tsv_path, split_name in [(train_tsv, 'train'), (test_tsv, 'test')]:
    with open(tsv_path, 'r', encoding='utf-8') as f:
        first_line = f.readline().strip()
        parts = first_line.split('\t')
        
        print(f"\n{split_name.upper()} TSV:")
        print(f"  Columns: {len(parts)}")
        print(f"  Expected: 3 (patient_id, left_img_b64, right_img_b64)")
        
        if len(parts) == 3:
            print(f"  ✅ Format correct")
            print(f"  Patient ID: {parts[0]}")
            print(f"  Left image length: {len(parts[1])} chars")
            print(f"  Right image length: {len(parts[2])} chars")
        else:
            print(f"  ❌ Format incorrect - expected 3 columns, got {len(parts)}")
            raise ValueError(f"Invalid TSV format for {split_name}")

print("\n✅ All TSV files validated successfully")

# STEP 4: Build LMDB Database

Create LMDB databases for efficient data loading during training.

In [ ]:
# Build LMDB for train set
print("="*80)
print("Building LMDB for TRAIN set")
print("="*80)

!python /content/retclip/RET_CLIP/preprocess/build_lmdb_dataset_for_RET-CLIP.py \
    --data_dir {DRIVE_DATA} \
    --splits train \
    --lmdb_dir {DRIVE_LMDB}

In [ ]:
# Build LMDB for test set
print("\n" + "="*80)
print("Building LMDB for TEST set")
print("="*80)

!python /content/retclip/RET_CLIP/preprocess/build_lmdb_dataset_for_RET-CLIP.py \
    --data_dir {DRIVE_DATA} \
    --splits test \
    --lmdb_dir {DRIVE_LMDB}

In [ ]:
# Validate LMDB by reading a few samples
import lmdb
import pickle

print("\n" + "="*80)
print("Validating LMDB databases")
print("="*80)

for split_name in ['train', 'test']:
    lmdb_path = f"{DRIVE_LMDB}/{split_name}/imgs"
    
    if not os.path.exists(lmdb_path):
        print(f"❌ LMDB not found: {lmdb_path}")
        continue
    
    env = lmdb.open(lmdb_path, readonly=True, lock=False, readahead=False, meminit=False)
    
    with env.begin() as txn:
        # Try to read first 3 samples
        print(f"\n{split_name.upper()} LMDB:")
        for i in range(3):
            patient_id = f"patient_{i:06d}"
            value = txn.get(patient_id.encode('utf-8'))
            
            if value is None:
                print(f"  ⚠️ No data for {patient_id}")
                continue
            
            try:
                img_left_b64, img_right_b64 = pickle.loads(value)
                print(f"  ✅ {patient_id}: left={len(img_left_b64)} chars, right={len(img_right_b64)} chars")
            except Exception as e:
                print(f"  ❌ Error unpacking {patient_id}: {e}")
    
    env.close()

print("\n✅ LMDB validation complete")

# STEP 5: Train RET-CLIP (10 Epochs)

Train RET-CLIP using contrastive learning on fundus images and clinical prompts.

In [ ]:
# Training configuration
# ⚠️ IMPORTANT: Batch size adjusted for T4 GPU (14.74 GiB VRAM)
# For T4: Use batch size 32-64 (128 is too large and causes OOM)
# For A100: Can use batch size 128-256
BATCH_SIZE = 32  # Reduced from 128 for T4 GPU compatibility
NUM_EPOCHS = NUM_EPOCHS_CONFIG  # Use config from cell-10 (2 for TEST_MODE, 10 for FULL_MODE)
LEARNING_RATE = 1e-5
WARMUP_STEPS = 500
VISION_MODEL = "ViT-B-16"
TEXT_MODEL = "microsoft-BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"

print("Training Configuration:")
print(f"  Mode: {'TEST' if TEST_MODE else 'FULL'}")
print(f"  GPU: Tesla T4 (14.74 GiB VRAM)")
print(f"  Batch size: {BATCH_SIZE} (optimized for T4)")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Warmup steps: {WARMUP_STEPS}")
print(f"  Vision model: {VISION_MODEL}")
print(f"  Text model: {TEXT_MODEL}")
print(f"  Train samples: {len(train_df)}")
print(f"  LMDB dir: {DRIVE_LMDB}/train")
print(f"  Checkpoint dir: {DRIVE_CHECKPOINTS}")
print(f"\n💡 Note: Batch size 32 prevents OOM on T4. Training will be slower but stable.")

In [ ]:
# Set PYTHONPATH for subprocess
import os
os.environ['PYTHONPATH'] = '/content/retclip'
print(f"✅ Set PYTHONPATH to: {os.environ['PYTHONPATH']}")

# Run training with distributed launcher
print("\n" + "="*80)
print("Starting RET-CLIP Training")
print("="*80)
print(f"\nThis will take approximately 6-8 hours on A100 GPU...\n")

!torchrun --nproc_per_node=1 --master_port=29500 \
    /content/retclip/RET_CLIP/training/main.py \
    --train-data {DRIVE_LMDB}/train \
    --batch-size {BATCH_SIZE} \
    --max-epochs {NUM_EPOCHS} \
    --lr {LEARNING_RATE} \
    --warmup {WARMUP_STEPS} \
    --vision-model {VISION_MODEL} \
    --text-model {TEXT_MODEL} \
    --logs {DRIVE_CHECKPOINTS} \
    --name retclip_fundus \
    --save-epoch-frequency 1 \
    --skip-aggregate

In [ ]:
# List saved checkpoints
print("\n" + "="*80)
print("Saved Checkpoints")
print("="*80)

# The training script creates a subdirectory structure: logs/name/checkpoints/
checkpoint_dir = f"{DRIVE_CHECKPOINTS}/retclip_fundus/checkpoints"

if os.path.exists(checkpoint_dir):
    checkpoint_files = sorted([f for f in os.listdir(checkpoint_dir) if f.endswith('.pt')])
    
    if checkpoint_files:
        print(f"\n✅ Found {len(checkpoint_files)} checkpoint(s) in {checkpoint_dir}:")
        for ckpt in checkpoint_files:
            ckpt_path = f"{checkpoint_dir}/{ckpt}"
            size_mb = os.path.getsize(ckpt_path) / (1024 * 1024)
            print(f"  {ckpt} ({size_mb:.1f} MB)")
    else:
        print(f"❌ No .pt files found in {checkpoint_dir}")
    
    # Verify final checkpoint exists (epoch2.pt for TEST_MODE, epoch_10.pt for FULL_MODE)
    final_checkpoint = f"{checkpoint_dir}/epoch{NUM_EPOCHS}.pt"
    if os.path.exists(final_checkpoint):
        print(f"\n✅ Final checkpoint ready: epoch{NUM_EPOCHS}.pt")
        print(f"   Path: {final_checkpoint}")
    else:
        print(f"\n⚠️ Final checkpoint not found: epoch{NUM_EPOCHS}.pt")
        print(f"   Expected: {final_checkpoint}")
else:
    print(f"❌ Checkpoint directory not found: {checkpoint_dir}")
    print(f"\nListing all files in {DRIVE_CHECKPOINTS}:")
    for item in os.listdir(DRIVE_CHECKPOINTS):
        print(f"  {item}")

## STEP 5B: Text Encoder Comparison (Optional)

### Overview

Compare different text encoders to evaluate which performs best for retinal disease classification:

1. **PubMedBERT** (medical domain-specific) - trained on PubMed abstracts
2. **Standard BERT** (general English) - trained on Wikipedia + BookCorpus  
3. **BioBERT** (biomedical domain) - BERT fine-tuned on PubMed + PMC articles

This allows us to quantify the benefit of domain-specific language models for medical image-text alignment.

**Note**: This step is optional but recommended for research purposes. It will train 3 separate RET-CLIP models (one for each text encoder), which will take ~18-24 hours total on A100 GPU.

If you want to skip this comparison and only use PubMedBERT, jump to Step 6.

In [ ]:
# Set PYTHONPATH for subprocess
import os
os.environ['PYTHONPATH'] = '/content/retclip'

# Train RET-CLIP with each text encoder (if comparison enabled)
if RUN_TEXT_ENCODER_COMPARISON:
    print("\n" + "="*80)
    print("Training RET-CLIP with Multiple Text Encoders")
    print("="*80)
    
    encoder_results = {}
    
    for encoder_config in TEXT_ENCODERS:
        encoder_name = encoder_config['name']
        encoder_model_id = encoder_config['model_id']
        
        print(f"\n{'='*80}")
        print(f"Training with {encoder_name}")
        print(f"{'='*80}")
        print(f"Model ID: {encoder_model_id}")
        print(f"Description: {encoder_config['description']}")
        print(f"\nEstimated time: ~6-8 hours on A100 GPU")
        print(f"Starting training...\n")
        
        # Create checkpoint directory for this encoder
        encoder_checkpoint_dir = f"{DRIVE_CHECKPOINTS}/{encoder_name.lower().replace('-', '_')}"
        os.makedirs(encoder_checkpoint_dir, exist_ok=True)
        
        # Run training with distributed launcher
        !torchrun --nproc_per_node=1 --master_port=29500 \
            /content/retclip/RET_CLIP/training/main.py \
            --train-data {DRIVE_LMDB}/train \
            --batch-size {BATCH_SIZE} \
            --max-epochs {NUM_EPOCHS} \
            --lr {LEARNING_RATE} \
            --warmup {WARMUP_STEPS} \
            --vision-model {VISION_MODEL} \
            --text-model {encoder_model_id} \
            --logs {encoder_checkpoint_dir} \
            --name retclip_{encoder_name.lower()} \
            --save-epoch-frequency 1 \
            --skip-aggregate
        
        # Store checkpoint path (with correct subdirectory structure)
        final_checkpoint = f"{encoder_checkpoint_dir}/retclip_{encoder_name.lower()}/checkpoints/epoch{NUM_EPOCHS}.pt"
        if os.path.exists(final_checkpoint):
            encoder_results[encoder_name] = {
                'checkpoint': final_checkpoint,
                'model_id': encoder_model_id,
                'status': 'success'
            }
            print(f"\n✅ {encoder_name} training complete")
            print(f"   Checkpoint: {final_checkpoint}")
        else:
            encoder_results[encoder_name] = {
                'checkpoint': None,
                'model_id': encoder_model_id,
                'status': 'failed'
            }
            print(f"\n❌ {encoder_name} training failed - checkpoint not found")
            print(f"   Expected: {final_checkpoint}")
    
    # Save encoder results
    import json
    with open(f"{DRIVE_RESULTS}/encoder_training_results.json", 'w') as f:
        json.dump(encoder_results, f, indent=2)
    
    print("\n" + "="*80)
    print("All encoder training complete!")
    print("="*80)
    
    for encoder_name, result in encoder_results.items():
        status_icon = "✅" if result['status'] == 'success' else "❌"
        print(f"{status_icon} {encoder_name}: {result['status']}")
        
else:
    print("\n⚠️ Skipping text encoder comparison training")
    print("   Only using PubMedBERT model from Step 5")
    
    # Store the single encoder result for later evaluation (with correct path structure)
    encoder_results = {
        "PubMedBERT": {
            'checkpoint': f"{DRIVE_CHECKPOINTS}/retclip_fundus/checkpoints/epoch{NUM_EPOCHS}.pt",
            'model_id': TEXT_MODEL,
            'status': 'success'
        }
    }

## Evaluate All Text Encoders

Perform zero-shot evaluation on all trained text encoders to compare performance.

In [ ]:
# NOTE: This cell should be run AFTER cell-29 (which creates encoder_results)
# Check if encoder_results exists (created by cell-29)
try:
    encoder_results
except NameError:
    print("⚠️ encoder_results not found - please run cell-29 first to train encoders")
    print("   Skipping encoder comparison evaluation")
    encoder_results = {}

# Evaluate all text encoders (if comparison enabled)
if RUN_TEXT_ENCODER_COMPARISON and len(encoder_results) > 0:
    print("\n" + "="*80)
    print("Evaluating All Text Encoders (Zero-Shot)")
    print("="*80)
    
    encoder_comparison_metrics = {}
    
    for encoder_name, encoder_info in encoder_results.items():
        if encoder_info['status'] != 'success':
            print(f"\n⚠️ Skipping {encoder_name} - training failed")
            continue
            
        print(f"\n{'='*80}")
        print(f"Evaluating {encoder_name}")
        print(f"{'='*80}")
        print(f"Model: {encoder_info['model_id']}")
        print(f"Checkpoint: {encoder_info['checkpoint']}")
        
        # Load model
        from RET_CLIP.clip.model import CLIP
        import json
        
        vision_config_path = f"/content/retclip/RET_CLIP/clip/model_configs/{VISION_MODEL}.json"
        text_config_path = f"/content/retclip/RET_CLIP/clip/model_configs/{encoder_info['model_id']}.json"
        
        with open(vision_config_path, 'r') as fv, open(text_config_path, 'r') as ft:
            model_cfg = json.load(fv)
            for k, v in json.load(ft).items():
                model_cfg[k] = v
        
        # Initialize model
        model = CLIP(**model_cfg)
        
        # Load checkpoint
        import torch
        checkpoint = torch.load(encoder_info['checkpoint'], map_location='cpu')
        state_dict = checkpoint['state_dict']
        
        # Handle DDP
        new_state_dict = {}
        for k, v in state_dict.items():
            name = k.replace('module.', '')
            new_state_dict[name] = v
        
        model.load_state_dict(new_state_dict)
        model = model.cuda()
        model.eval()
        
        print(f"✅ Model loaded")
        
        # Prepare prompts
        disease_classes = test_df['label'].unique().tolist()
        zero_shot_prompts = {}
        for disease in disease_classes:
            sample_prompt = test_df[test_df['label'] == disease]['prompt'].iloc[0]
            zero_shot_prompts[disease] = sample_prompt
        
        # Encode text
        from RET_CLIP.clip import tokenize
        
        disease_list = list(zero_shot_prompts.keys())
        prompt_list = [zero_shot_prompts[d] for d in disease_list]
        
        text_tokens = tokenize(prompt_list).cuda()
        
        with torch.no_grad():
            text_features = model.encode_text(text_tokens)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        
        print(f"✅ Text features encoded: {text_features.shape}")
        
        # Image preprocessing
        from torchvision import transforms
        from PIL import Image
        from io import BytesIO
        import base64
        import pickle
        import lmdb
        from tqdm.notebook import tqdm
        
        transform = transforms.Compose([
            transforms.Resize(IMAGE_SIZE, interpolation=transforms.InterpolationMode.BICUBIC),
            transforms.CenterCrop(IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=(0.48145466, 0.4578275, 0.40821073),
                std=(0.26862954, 0.26130258, 0.27577711)
            )
        ])
        
        # Zero-shot inference
        test_lmdb_path = f"{DRIVE_LMDB}/test"
        env = lmdb.open(test_lmdb_path, readonly=True, lock=False, readahead=False, meminit=False)
        
        predictions = []
        ground_truth = []
        
        with env.begin() as txn:
            for idx in tqdm(range(len(test_df)), desc=f"  {encoder_name} inference"):
                try:
                    patient_id = f"patient_{idx:06d}"
                    value = txn.get(patient_id.encode('utf-8'))
                    
                    if value is None:
                        continue
                    
                    img_left_b64, img_right_b64 = pickle.loads(value)
                    img_bytes = base64.urlsafe_b64decode(img_left_b64)
                    image = Image.open(BytesIO(img_bytes)).convert('RGB')
                    
                    img_tensor = transform(image).unsqueeze(0).cuda()
                    
                    with torch.no_grad():
                        img_features = model.encode_image(img_tensor)
                        img_features = img_features / img_features.norm(dim=-1, keepdim=True)
                    
                    similarities = (img_features @ text_features.T).squeeze(0)
                    pred_idx = similarities.argmax().item()
                    predicted_disease = disease_list[pred_idx]
                    
                    true_disease = test_df.iloc[idx]['label']
                    
                    predictions.append(predicted_disease)
                    ground_truth.append(true_disease)
                    
                except Exception as e:
                    continue
        
        env.close()
        
        # Compute metrics
        from sklearn.metrics import accuracy_score, classification_report, f1_score
        
        accuracy = accuracy_score(ground_truth, predictions)
        macro_f1 = f1_score(ground_truth, predictions, average='macro')
        weighted_f1 = f1_score(ground_truth, predictions, average='weighted')
        
        report = classification_report(ground_truth, predictions, 
                                      target_names=disease_list, output_dict=True)
        
        encoder_comparison_metrics[encoder_name] = {
            'accuracy': float(accuracy),
            'macro_f1': float(macro_f1),
            'weighted_f1': float(weighted_f1),
            'per_class': report
        }
        
        print(f"\n✅ {encoder_name} Results:")
        print(f"   Accuracy: {accuracy * 100:.2f}%")
        print(f"   Macro F1: {macro_f1 * 100:.2f}%")
        print(f"   Weighted F1: {weighted_f1 * 100:.2f}%")
        
        # Clean up
        del model
        torch.cuda.empty_cache()
    
    # Save comparison metrics
    import json
    with open(f"{DRIVE_RESULTS}/encoder_comparison_metrics.json", 'w') as f:
        json.dump(encoder_comparison_metrics, f, indent=2)
    
    print("\n" + "="*80)
    print("Text Encoder Comparison Summary")
    print("="*80)
    
    # Create comparison table
    import pandas as pd
    comparison_data = []
    for encoder_name, metrics in encoder_comparison_metrics.items():
        comparison_data.append({
            'Text Encoder': encoder_name,
            'Accuracy': f"{metrics['accuracy'] * 100:.2f}%",
            'Macro F1': f"{metrics['macro_f1'] * 100:.2f}%",
            'Weighted F1': f"{metrics['weighted_f1'] * 100:.2f}%"
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    display(comparison_df)
    
    # Save comparison
    comparison_df.to_csv(f"{DRIVE_RESULTS}/encoder_comparison_table.csv", index=False)
    
    print(f"\n✅ Comparison saved to {DRIVE_RESULTS}/encoder_comparison_table.csv")
    
    # Determine best encoder
    best_encoder = max(encoder_comparison_metrics.items(), 
                      key=lambda x: x[1]['accuracy'])
    print(f"\n🏆 Best Text Encoder: {best_encoder[0]}")
    print(f"   Accuracy: {best_encoder[1]['accuracy'] * 100:.2f}%")
    
else:
    if not RUN_TEXT_ENCODER_COMPARISON:
        print("\n⚠️ Text encoder comparison disabled")
        print(f"   Set RUN_TEXT_ENCODER_COMPARISON = True in cell-10 to enable")
    else:
        print("\n⚠️ No encoder results available - run cell-29 first")
        print(f"   Using single encoder mode (PubMedBERT only)")

# STEP 6: Zero-Shot Evaluation

Test RET-CLIP's vision-language alignment by computing similarity between image embeddings and text embeddings for all disease classes.

In [ ]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import json

# Load RET-CLIP model
print("Loading RET-CLIP model...")

from RET_CLIP.clip.model import CLIP

# Load model configs
vision_config_path = f"/content/retclip/RET_CLIP/clip/model_configs/{VISION_MODEL}.json"
text_config_path = f"/content/retclip/RET_CLIP/clip/model_configs/{TEXT_MODEL}.json"

with open(vision_config_path, 'r') as fv, open(text_config_path, 'r') as ft:
    model_cfg = json.load(fv)
    for k, v in json.load(ft).items():
        model_cfg[k] = v

# Initialize model
model = CLIP(**model_cfg)

# Load trained checkpoint (uses correct path based on training structure)
checkpoint_path = f"{DRIVE_CHECKPOINTS}/retclip_fundus/checkpoints/epoch{NUM_EPOCHS}.pt"
print(f"Loading checkpoint from: {checkpoint_path}")

checkpoint = torch.load(checkpoint_path, map_location='cpu')

# Handle DDP state dict (strip 'module.' prefix)
state_dict = checkpoint['state_dict']
new_state_dict = {}
for k, v in state_dict.items():
    name = k.replace('module.', '')
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)
model = model.cuda()
model.eval()

print("✅ Model loaded successfully")

In [ ]:
# Prepare zero-shot prompts for all disease classes
# Use diverse prompts from the test set for each class

disease_classes = test_df['label'].unique().tolist()
print(f"Disease classes: {len(disease_classes)}")
print(disease_classes)

# Get one representative prompt per disease class
zero_shot_prompts = {}
for disease in disease_classes:
    # Get a random prompt for this disease
    sample_prompt = test_df[test_df['label'] == disease]['prompt'].iloc[0]
    zero_shot_prompts[disease] = sample_prompt

print("\nZero-shot prompts:")
for disease, prompt in zero_shot_prompts.items():
    print(f"  {disease}: {prompt[:80]}...")

In [ ]:
# Encode text prompts
from RET_CLIP.clip import tokenize

print("\nEncoding text prompts...")

disease_list = list(zero_shot_prompts.keys())
prompt_list = [zero_shot_prompts[d] for d in disease_list]

# Tokenize and encode
text_tokens = tokenize(prompt_list).cuda()

with torch.no_grad():
    text_features = model.encode_text(text_tokens)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

print(f"✅ Text features shape: {text_features.shape}")
print(f"   {len(disease_list)} disease classes encoded")

In [ ]:
# Extract image features from test set and perform zero-shot classification
from torchvision import transforms
from PIL import Image
from io import BytesIO
import base64
import pickle
import lmdb
from tqdm.notebook import tqdm

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.48145466, 0.4578275, 0.40821073),
        std=(0.26862954, 0.26130258, 0.27577711)
    )
])

# Load test LMDB
test_lmdb_path = f"{DRIVE_LMDB}/test"
env = lmdb.open(test_lmdb_path, readonly=True, lock=False, readahead=False, meminit=False)

print("\nPerforming zero-shot classification on test set...")

predictions = []
ground_truth = []

with env.begin() as txn:
    for idx in tqdm(range(len(test_df)), desc="Zero-shot inference"):
        try:
            # Get image from LMDB
            patient_id = f"patient_{idx:06d}"
            value = txn.get(patient_id.encode('utf-8'))
            
            if value is None:
                continue
            
            # Decode image
            img_left_b64, img_right_b64 = pickle.loads(value)
            img_bytes = base64.urlsafe_b64decode(img_left_b64)
            image = Image.open(BytesIO(img_bytes)).convert('RGB')
            
            # Transform and encode image
            img_tensor = transform(image).unsqueeze(0).cuda()
            
            with torch.no_grad():
                img_features = model.encode_image(img_tensor)
                img_features = img_features / img_features.norm(dim=-1, keepdim=True)
            
            # Compute similarity with all disease prompts
            similarities = (img_features @ text_features.T).squeeze(0)
            
            # Predict class
            pred_idx = similarities.argmax().item()
            predicted_disease = disease_list[pred_idx]
            
            # Get ground truth
            true_disease = test_df.iloc[idx]['label']
            
            predictions.append(predicted_disease)
            ground_truth.append(true_disease)
            
        except Exception as e:
            print(f"Error at index {idx}: {e}")
            continue

env.close()

print(f"\n✅ Zero-shot inference complete")
print(f"   Processed {len(predictions)} samples")

In [ ]:
# Compute zero-shot metrics
print("\n" + "="*80)
print("ZERO-SHOT EVALUATION RESULTS")
print("="*80)

# Accuracy
zeroshot_acc = accuracy_score(ground_truth, predictions)
print(f"\nZero-Shot Accuracy: {zeroshot_acc * 100:.2f}%")

# Classification report
print("\nPer-Class Metrics:")
report = classification_report(ground_truth, predictions, target_names=disease_list, output_dict=True)
print(classification_report(ground_truth, predictions, target_names=disease_list))

# Save metrics
zeroshot_metrics = {
    'accuracy': float(zeroshot_acc),
    'per_class': report
}

with open(f"{DRIVE_RESULTS}/zeroshot_metrics.json", 'w') as f:
    json.dump(zeroshot_metrics, f, indent=2)

print(f"\n✅ Metrics saved to {DRIVE_RESULTS}/zeroshot_metrics.json")

In [ ]:
# Plot confusion matrix
cm = confusion_matrix(ground_truth, predictions, labels=disease_list)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=disease_list, yticklabels=disease_list)
plt.title('Zero-Shot Confusion Matrix', fontsize=16, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(f"{DRIVE_RESULTS}/zeroshot_confusion_matrix.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Confusion matrix saved to {DRIVE_RESULTS}/zeroshot_confusion_matrix.png")

# STEP 7: Linear Probing Evaluation

Train a linear classifier on frozen RET-CLIP features to evaluate feature quality.

In [ ]:
# Extract features from train set
print("="*80)
print("Extracting features from TRAIN set")
print("="*80)

train_lmdb_path = f"{DRIVE_LMDB}/train"
env = lmdb.open(train_lmdb_path, readonly=True, lock=False, readahead=False, meminit=False)

train_features = []
train_labels = []

# Create label mapping
label_to_idx = {label: idx for idx, label in enumerate(disease_list)}

with env.begin() as txn:
    for idx in tqdm(range(len(train_df)), desc="Extracting train features"):
        try:
            patient_id = f"patient_{idx:06d}"
            value = txn.get(patient_id.encode('utf-8'))
            
            if value is None:
                continue
            
            # Decode image
            img_left_b64, img_right_b64 = pickle.loads(value)
            img_bytes = base64.urlsafe_b64decode(img_left_b64)
            image = Image.open(BytesIO(img_bytes)).convert('RGB')
            
            # Transform and extract features
            img_tensor = transform(image).unsqueeze(0).cuda()
            
            with torch.no_grad():
                img_features = model.encode_image(img_tensor)
                img_features = img_features / img_features.norm(dim=-1, keepdim=True)
            
            train_features.append(img_features.cpu().numpy())
            
            # Get label
            label = train_df.iloc[idx]['label']
            train_labels.append(label_to_idx[label])
            
        except Exception as e:
            print(f"Error at index {idx}: {e}")
            continue

env.close()

train_features = np.vstack(train_features)
train_labels = np.array(train_labels)

print(f"\n✅ Extracted train features: {train_features.shape}")
print(f"   Labels: {train_labels.shape}")

# Save features
np.save(f"{DRIVE_RESULTS}/train_features.npy", train_features)
np.save(f"{DRIVE_RESULTS}/train_labels.npy", train_labels)
print(f"   Saved to {DRIVE_RESULTS}/train_features.npy")

In [ ]:
# Extract features from test set
print("\n" + "="*80)
print("Extracting features from TEST set")
print("="*80)

test_lmdb_path = f"{DRIVE_LMDB}/test"
env = lmdb.open(test_lmdb_path, readonly=True, lock=False, readahead=False, meminit=False)

test_features = []
test_labels = []

with env.begin() as txn:
    for idx in tqdm(range(len(test_df)), desc="Extracting test features"):
        try:
            patient_id = f"patient_{idx:06d}"
            value = txn.get(patient_id.encode('utf-8'))
            
            if value is None:
                continue
            
            # Decode image
            img_left_b64, img_right_b64 = pickle.loads(value)
            img_bytes = base64.urlsafe_b64decode(img_left_b64)
            image = Image.open(BytesIO(img_bytes)).convert('RGB')
            
            # Transform and extract features
            img_tensor = transform(image).unsqueeze(0).cuda()
            
            with torch.no_grad():
                img_features = model.encode_image(img_tensor)
                img_features = img_features / img_features.norm(dim=-1, keepdim=True)
            
            test_features.append(img_features.cpu().numpy())
            
            # Get label
            label = test_df.iloc[idx]['label']
            test_labels.append(label_to_idx[label])
            
        except Exception as e:
            print(f"Error at index {idx}: {e}")
            continue

env.close()

test_features = np.vstack(test_features)
test_labels = np.array(test_labels)

print(f"\n✅ Extracted test features: {test_features.shape}")
print(f"   Labels: {test_labels.shape}")

# Save features
np.save(f"{DRIVE_RESULTS}/test_features.npy", test_features)
np.save(f"{DRIVE_RESULTS}/test_labels.npy", test_labels)
print(f"   Saved to {DRIVE_RESULTS}/test_features.npy")

In [ ]:
# Train logistic regression classifier
from sklearn.linear_model import LogisticRegression
import pickle as pkl

print("\n" + "="*80)
print("Training Linear Probe Classifier")
print("="*80)

# Train classifier
classifier = LogisticRegression(
    max_iter=1000,
    multi_class='multinomial',
    random_state=42,
    verbose=1
)

classifier.fit(train_features, train_labels)

print("\n✅ Classifier trained successfully")

# Save classifier
with open(f"{DRIVE_RESULTS}/linear_probe_classifier.pkl", 'wb') as f:
    pkl.dump(classifier, f)

print(f"   Saved to {DRIVE_RESULTS}/linear_probe_classifier.pkl")

In [ ]:
# Evaluate classifier
print("\n" + "="*80)
print("LINEAR PROBING EVALUATION RESULTS")
print("="*80)

# Predict on test set
linear_probe_preds = classifier.predict(test_features)

# Accuracy
linear_probe_acc = accuracy_score(test_labels, linear_probe_preds)
print(f"\nLinear Probe Accuracy: {linear_probe_acc * 100:.2f}%")

# Classification report
print("\nPer-Class Metrics:")
linear_report = classification_report(test_labels, linear_probe_preds, 
                                     target_names=disease_list, output_dict=True)
print(classification_report(test_labels, linear_probe_preds, target_names=disease_list))

# Save metrics
linear_probe_metrics = {
    'accuracy': float(linear_probe_acc),
    'per_class': linear_report
}

with open(f"{DRIVE_RESULTS}/linear_probe_metrics.json", 'w') as f:
    json.dump(linear_probe_metrics, f, indent=2)

print(f"\n✅ Metrics saved to {DRIVE_RESULTS}/linear_probe_metrics.json")

In [ ]:
# Plot confusion matrix
cm = confusion_matrix(test_labels, linear_probe_preds)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', 
            xticklabels=disease_list, yticklabels=disease_list)
plt.title('Linear Probing Confusion Matrix', fontsize=16, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(f"{DRIVE_RESULTS}/linear_probe_confusion_matrix.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Confusion matrix saved to {DRIVE_RESULTS}/linear_probe_confusion_matrix.png")

# STEP 8: Generate Final Report

Consolidate all metrics and create comparison report.

In [ ]:
# Create comparison table
print("="*80)
print("FINAL RESULTS COMPARISON")
print("="*80)

comparison_df = pd.DataFrame({
    'Method': ['Zero-Shot', 'Linear Probing'],
    'Accuracy': [
        f"{zeroshot_acc * 100:.2f}%",
        f"{linear_probe_acc * 100:.2f}%"
    ],
    'Macro F1': [
        f"{report['macro avg']['f1-score'] * 100:.2f}%",
        f"{linear_report['macro avg']['f1-score'] * 100:.2f}%"
    ],
    'Weighted F1': [
        f"{report['weighted avg']['f1-score'] * 100:.2f}%",
        f"{linear_report['weighted avg']['f1-score'] * 100:.2f}%"
    ]
})

print("\n")
display(comparison_df)

# Save comparison
comparison_df.to_csv(f"{DRIVE_RESULTS}/metrics_comparison.csv", index=False)
print(f"\n✅ Saved to {DRIVE_RESULTS}/metrics_comparison.csv")

In [ ]:
# Generate final report
final_report = f"""
{'='*80}
RET-CLIP TRAINING & EVALUATION REPORT
{'='*80}

Date: {datetime.datetime.now()}

DATASET STATISTICS
{'-'*80}
Train samples: {len(train_df)}
Test samples: {len(test_df)}
Disease classes: {len(disease_list)}
Classes: {', '.join(disease_list)}

TRAINING CONFIGURATION
{'-'*80}
Vision Model: {VISION_MODEL}
Text Model: {TEXT_MODEL}
Batch Size: {BATCH_SIZE}
Epochs: {NUM_EPOCHS}
Learning Rate: {LEARNING_RATE}
Image Size: {IMAGE_SIZE}x{IMAGE_SIZE}

EVALUATION RESULTS
{'-'*80}

1. Zero-Shot Evaluation (Vision-Language Alignment)
   Accuracy: {zeroshot_acc * 100:.2f}%
   Macro F1: {report['macro avg']['f1-score'] * 100:.2f}%
   Weighted F1: {report['weighted avg']['f1-score'] * 100:.2f}%

2. Linear Probing (Feature Quality)
   Accuracy: {linear_probe_acc * 100:.2f}%
   Macro F1: {linear_report['macro avg']['f1-score'] * 100:.2f}%
   Weighted F1: {linear_report['weighted avg']['f1-score'] * 100:.2f}%

ARTIFACTS SAVED TO GOOGLE DRIVE
{'-'*80}
{DRIVE_BASE}/
├── data/
│   ├── train_imgs.tsv
│   ├── test_imgs.tsv
│   ├── train_texts.jsonl
│   └── test_texts.jsonl
├── lmdb/
│   ├── train/
│   └── test/
├── checkpoints/
│   └── epoch_10.pt
└── results/
    ├── zeroshot_metrics.json
    ├── zeroshot_confusion_matrix.png
    ├── linear_probe_metrics.json
    ├── linear_probe_confusion_matrix.png
    ├── linear_probe_classifier.pkl
    ├── train_features.npy
    ├── test_features.npy
    ├── metrics_comparison.csv
    └── final_report.txt

{'='*80}
END OF REPORT
{'='*80}
"""

print(final_report)

# Save report
with open(f"{DRIVE_RESULTS}/final_report.txt", 'w') as f:
    f.write(final_report)

print(f"\n✅ Final report saved to {DRIVE_RESULTS}/final_report.txt")

In [ ]:
# List all generated artifacts
print("\n" + "="*80)
print("ALL ARTIFACTS SAVED TO GOOGLE DRIVE")
print("="*80)

def list_files_recursive(path, prefix=""):
    """List files recursively with tree structure"""
    items = sorted(os.listdir(path))
    for i, item in enumerate(items):
        item_path = os.path.join(path, item)
        is_last = (i == len(items) - 1)
        
        connector = "└── " if is_last else "├── "
        
        if os.path.isdir(item_path):
            print(f"{prefix}{connector}{item}/")
            extension = "    " if is_last else "│   "
            list_files_recursive(item_path, prefix + extension)
        else:
            size_mb = os.path.getsize(item_path) / (1024 * 1024)
            print(f"{prefix}{connector}{item} ({size_mb:.1f} MB)")

print(f"\n{DRIVE_BASE}/")
list_files_recursive(DRIVE_BASE)

print("\n" + "="*80)
print("✅ PIPELINE COMPLETE!")
print("="*80)
print(f"\nAll artifacts saved to: {DRIVE_BASE}")
print(f"\nYou can now:")
print(f"  1. Review metrics in {DRIVE_RESULTS}/final_report.txt")
print(f"  2. Use the trained model: {DRIVE_CHECKPOINTS}/epoch_10.pt")
print(f"  3. Download artifacts from Google Drive for local analysis")